In [1]:
import pandas as pd 
import numpy as np

In [2]:
all_data_test = pd.read_csv('all_data_test.csv')

In [3]:
all_data_test.shape

(522341, 136)

In [4]:
all_data_test.sample()

,user_id,merchant_id,label,prob,age_range,gender,item_path,cat_path,seller_path,brand_path,...,tfidf_90,tfidf_91,tfidf_92,tfidf_93,tfidf_94,tfidf_95,tfidf_96,tfidf_97,tfidf_98,tfidf_99
184977,160673,2241,0.0,NaN,2.0,0.0,729175 544824 273795 544824 930806 930806 2737...,407 555 1326 555 555 555 1326 555 1213 602 602...,2709 1783 801 1783 2278 2278 801 1783 1760 176...,4360.0 5041.0 5890.0 5041.0 6257.0 6257.0 5890...,...,0.0,0.0,0.0,0.0,0.059968,0.0,0.0,0.0,0.0,0.0


In [5]:
feature_columns = [col for col in all_data_test.columns if col not in ['prob','item_path','cat_path',\
                                                              'seller_path','brand_path','time_stamp_path',\
                                                                'action_type_path']]

x_train = all_data_test[~all_data_test['label'].isna()][feature_columns].values
y_train = all_data_test[all_data_test['label'].isna()][feature_columns].values
x_valid = all_data_test[all_data_test['label'].isna()][feature_columns].values


In [6]:
feature_columns

['user_id',
 'merchant_id',
 'label',
 'age_range',
 'gender',
 'user_cnt',
 'seller_nunique',
 'cat_nunique',
 'brand_nunique',
 'item_nunique',
 'time_stamp_nunique',
 'action_type_nunique',
 'time_stamp_max',
 'time_stamp_min',
 'time_stamp_std',
 'time_stamp_range',
 'seller_most_1',
 'cat_most_1',
 'brand_most_1',
 'action_type_1',
 'seller_most_1_cnt',
 'cat_most_1_cnt',
 'brand_most_1_cnt',
 'action_type_most_1_cnt',
 'user_cnt_0',
 'user_cnt_1',
 'user_cnt_2',
 'user_cnt_3',
 'seller_nunique_0',
 'tfidf_0',
 'tfidf_1',
 'tfidf_2',
 'tfidf_3',
 'tfidf_4',
 'tfidf_5',
 'tfidf_6',
 'tfidf_7',
 'tfidf_8',
 'tfidf_9',
 'tfidf_10',
 'tfidf_11',
 'tfidf_12',
 'tfidf_13',
 'tfidf_14',
 'tfidf_15',
 'tfidf_16',
 'tfidf_17',
 'tfidf_18',
 'tfidf_19',
 'tfidf_20',
 'tfidf_21',
 'tfidf_22',
 'tfidf_23',
 'tfidf_24',
 'tfidf_25',
 'tfidf_26',
 'tfidf_27',
 'tfidf_28',
 'tfidf_29',
 'tfidf_30',
 'tfidf_31',
 'tfidf_32',
 'tfidf_33',
 'tfidf_34',
 'tfidf_35',
 'tfidf_36',
 'tfidf_37',
 'tfidf

In [7]:
def get_martix(data):
    where_are_nan = np.isnan(data)
    where_are_inf = np.isinf(data)
    data[where_are_nan] = 0
    data[where_are_inf] = 0
    return data

x_train = np.float_(get_martix(np.float_(x_train)))
y_train = np.int_(y_train)
#x_valid = x_train
x_valid = np.float_(get_martix(np.float_(x_valid)))

In [8]:
all_data_test.shape

(522341, 136)

In [9]:
x_train.shape

(260864, 129)

In [56]:
x_valid.shape

(261477, 129)

In [57]:
test_data.shape

(261477, 129)

In [10]:
train_data = pd.DataFrame(x_train)
train_data.columns = feature_columns

test_data = pd.DataFrame(x_valid)
test_data.columns = feature_columns

# 模型验证

In [11]:
train_data.sample()

,user_id,merchant_id,label,age_range,gender,user_cnt,seller_nunique,cat_nunique,brand_nunique,item_nunique,...,tfidf_90,tfidf_91,tfidf_92,tfidf_93,tfidf_94,tfidf_95,tfidf_96,tfidf_97,tfidf_98,tfidf_99
42657,178686.0,1618.0,1.0,0.0,0.0,104.0,66.0,39.0,66.0,74.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
feature_columns = [col for col in train_data.columns if col not in ['label','user_id']]

train = train_data[feature_columns].values
test = train_data[feature_columns].values
target = train_data['label'].values

## 基础代码验证

In [13]:
# 40% 数据用于线下验证

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100,max_depth=2,random_state=0,n_jobs=-1)

X_train,X_test,y_train,y_test = train_test_split(train,target,test_size=0.4,random_state=0)

print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

clf = clf.fit(X_train,y_train)
clf.score(X_test,y_test)


(156518, 127) (156518,)
(104346, 127) (104346,)


0.9387710118260403

## 交叉验证

In [15]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100,max_depth=2,random_state=0,n_jobs=-1)
scores = cross_val_score(clf,train,target,cv=5)
print(scores)
print('Accuracy: %0.2f (+/- %0.2f)' % (scores.mean(),scores.std()*2))


[0.93883927 0.93883927 0.93885609 0.93885609 0.93885609]
Accuracy: 0.94 (+/- 0.00)


### 用F1评价模型

In [16]:
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier


clf = RandomForestClassifier(n_estimators=100,max_depth=2,random_state=0,n_jobs=-1)
scores = cross_val_score(clf,train,target,cv=5,scoring='f1_macro')

print(scores)
print('F1: %0.2f (+/- %0.2f)' % (scores.mean(),scores.std()*2))

C:\SoftDir\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\SoftDir\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\SoftDir\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\SoftDir\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[0.48422749 0.48422749 0.48423196 0.48423196 0.48423196]
F1: 0.48 (+/- 0.00)


C:\SoftDir\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 设置交叉验证方式

#### 使用ShuffleSplit切分数据

In [17]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100,max_depth=2,random_state=0,n_jobs=-1)
cv = ShuffleSplit(n_splits=5,test_size=0.3,random_state=0)

cross_val_score(clf,train,target,cv=cv)

array([0.93870432, 0.93828265, 0.93956044, 0.94040378, 0.93763097])

#### 使用KFlod切分数据

In [18]:
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100,max_depth=2,random_state=0,n_jobs=-1)
kf = KFold(n_splits=5)

for k,(train_index,test_index) in enumerate(kf.split(train)):
    X_train,X_test,y_train,y_test = train[train_index],train[test_index],target[train_index],target[test_index]
    clf = clf.fit(X_train,y_train)
    print(k,clf.score(X_test,y_test))

0 0.9395089414064746
1 0.9386272593103713
2 0.9374005711766623
3 0.9398539474440802
4 0.9388560913900177


####  使用ShuffleKFlod切分数据(label均分)

In [19]:
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100,max_depth=2,random_state=0,n_jobs=-1)
skf = StratifiedKFold(n_splits=5)


for k,(train_index,test_index) in enumerate(skf.split(train,target)):
    X_train,X_test,y_train,y_test = train[train_index],train[test_index],target[train_index],target[test_index]
    clf = clf.fit(X_train,y_train)
    print(k,clf.score(X_test,y_test))

0 0.938839268601219
1 0.938839268601219
2 0.9388560913900177
3 0.9388560913900177
4 0.9388560913900177


# 模型调参

In [20]:
# 对模型调参，然后预测并评估模型性能

In [21]:
'''
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier


# split the data into two equal parts
X_train,X_test,y_train,y_test = train_test_split(train,target,test_size=0.5,random_state=0)

# model
clf = RandomForestClassifier(n_jobs=-1)

# set the parameters by cross-validation

tuned_parameters = {'n_estimators':[50,100,200]}

scores = ['precision']

for score in scores:
    print('# Tuning hyper-parameters for %s' %score)
    
    clf = GridSearchCV(clf,tuned_parameters,cv=5,scores='%s_macro'%score)
    clf.fit(X_train,y_train)
    
    print('Best parameters set found on development set:')
    
    print(clf.best_params_)
    
'''



"\nfrom sklearn.model_selection import train_test_split\nfrom sklearn.model_selection import GridSearchCV\nfrom sklearn.metrics import classification_report\nfrom sklearn.ensemble import RandomForestClassifier\n\n\n# split the data into two equal parts\nX_train,X_test,y_train,y_test = train_test_split(train,target,test_size=0.5,random_state=0)\n\n# model\nclf = RandomForestClassifier(n_jobs=-1)\n\n# set the parameters by cross-validation\n\ntuned_parameters = {'n_estimators':[50,100,200]}\n\nscores = ['precision']\n\nfor score in scores:\n    print('# Tuning hyper-parameters for %s' %score)\n    \n    clf = GridSearchCV(clf,tuned_parameters,cv=5,scores='%s_macro'%score)\n    clf.fit(X_train,y_train)\n    \n    print('Best parameters set found on development set:')\n    \n    print(clf.best_params_)\n    \n"

# 不同分类模型

## LR模型

In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

stdScaler = StandardScaler()
X = stdScaler.fit_transform(train)


X_train,X_test,y_train,y_test = train_test_split(X,target,random_state=0)
clf = LogisticRegression(random_state=0,solver='lbfgs',multi_class='multinomial')
clf.fit(X_train,y_train)
clf.score(X_test,y_test)



0.9386653581943082

## KN模型

In [24]:
'''
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

stdScaler = StandardScaler()
X = stdScaler.fit_transform(train)


X_train,X_test,y_train,y_test = train_test_split(X,target,random_state=0)
clf = KNeighborsClassifier(n_neighbors=3)
clf.fit(X_train,y_train)
clf.score(X_test,y_test)
'''

'\nfrom sklearn.neighbors import KNeighborsClassifier\nfrom sklearn.preprocessing import StandardScaler\nfrom sklearn.model_selection import train_test_split\n\nstdScaler = StandardScaler()\nX = stdScaler.fit_transform(train)\n\n\nX_train,X_test,y_train,y_test = train_test_split(X,target,random_state=0)\nclf = KNeighborsClassifier(n_neighbors=3)\nclf.fit(X_train,y_train)\nclf.score(X_test,y_test)\n'

## 高斯贝叶斯模型

In [25]:
from sklearn.navie_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

stdScaler = StandardScaler()
X = stdScaler.fit_transform(train)


X_train,X_test,y_train,y_test = train_test_split(X,target,random_state=0)
clf = GaussianNB()
clf.fit(X_train,y_train)
clf.score(X_test,y_test)


ModuleNotFoundError: No module named 'sklearn.navie_bayes'

### Lgb模型

In [59]:
all_data_test.sample()

,user_id,merchant_id,label,prob,age_range,gender,item_path,cat_path,seller_path,brand_path,...,tfidf_90,tfidf_91,tfidf_92,tfidf_93,tfidf_94,tfidf_95,tfidf_96,tfidf_97,tfidf_98,tfidf_99
38143,292848,4760,0.0,NaN,3.0,1.0,1112396 313504 39268 192201 904910 192201 9049...,1421 1421 1326 407 1438 407 1438 1213 1213 276...,2273 749 308 3936 3936 3936 3936 1200 1200 361...,2240.0 2630.0 1077.0 5430.0 229.0 229.0 5430.0...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [90]:
initial_feature_columns = [col for col in all_data_test.columns if col not in ['prob','item_path','cat_path',\
                                                              'seller_path','brand_path','time_stamp_path',\
                                                                'action_type_path']]

feature_columns = [col for col in initial_feature_columns if col not in ['label','user_id']]


Train_data = all_data_test[~all_data_test['label'].isna()][feature_columns]

X_data = Train_data[feature_columns].values

Y_data = all_data_test[~all_data_test['label'].isna()]['label'].values


X_test = all_data_test[all_data_test['label'].isna()][feature_columns].values


In [104]:
from sklearn.model_selection import KFold
import lightgbm as lgb 

params = {'num_leaves': 60, #结果对最终效果影响较大，越大值越好，太大会出现过拟合
          'min_data_in_leaf': 30,
          'objective': 'binary', #定义的目标函数
          'max_depth': -1,
          'learning_rate': 0.03,
          "min_sum_hessian_in_leaf": 6,
          "boosting": "gbdt",
          "feature_fraction": 0.9,  #提取的特征比率
          "bagging_freq": 1,
          "bagging_fraction": 0.8,
          "bagging_seed": 11,
          "lambda_l1": 0.1,             #l1正则
          # 'lambda_l2': 0.001,     #l2正则
          "verbosity": -1,
          "nthread": -1,                #线程数量，-1表示全部线程，线程越多，运行的速度越快
          'metric': {'binary_logloss', 'auc'},  ##评价函数选择
          #"random_state": 2019, #随机数种子，可以防止每次运行的结果不一致
          # 'device': 'gpu' ##如果安装的事gpu版本的lightgbm,可以加快运算
          }

folds = KFold(n_splits=5,shuffle=True,random_state=2019)
prob_oof = np.zeros(len(X_data))
predictions_test_lgb = np.zeros(len(X_test))
predictions_train_lgb = np.zeros(len(X_data))


## train and predict
feature_importance_df = pd.DataFrame()
for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_data,Y_data)):
    print("fold {}".format(fold_ + 1))
    trn_data = lgb.Dataset(X_data[trn_idx], label=Y_data[trn_idx])
    val_data = lgb.Dataset(X_data[val_idx], label=Y_data[val_idx])

    num_round = 1000
    clf = lgb.train(params,
                    trn_data,
                    num_round,
                    valid_sets=[trn_data, val_data],
                    verbose_eval=300,
                    early_stopping_rounds=200)
    prob_oof[val_idx] = clf.predict(X_data[val_idx], num_iteration=clf.best_iteration)

    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = feature_columns
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    predictions_test_lgb += clf.predict(X_test, num_iteration=clf.best_iteration) / folds.n_splits
    predictions_train_lgb += clf.predict(X_data,num_iteration=clf.best_iteration) / folds.n_splits
    
    

fold 1
Train until valid scores didn't improve in 200 rounds.
[300]	training's auc: 0.781764	training's binary_logloss: 0.205918	valid_1's auc: 0.61411	valid_1's binary_logloss: 0.22283
Early stopping, best iteration is:
[261]	training's auc: 0.765695	training's binary_logloss: 0.208557	valid_1's auc: 0.615795	valid_1's binary_logloss: 0.22279
fold 2
Train until valid scores didn't improve in 200 rounds.
[300]	training's auc: 0.778544	training's binary_logloss: 0.205232	valid_1's auc: 0.607103	valid_1's binary_logloss: 0.227025
Early stopping, best iteration is:
[248]	training's auc: 0.756758	training's binary_logloss: 0.208697	valid_1's auc: 0.608146	valid_1's binary_logloss: 0.226978
fold 3
Train until valid scores didn't improve in 200 rounds.
[300]	training's auc: 0.777424	training's binary_logloss: 0.205135	valid_1's auc: 0.611126	valid_1's binary_logloss: 0.22733
Early stopping, best iteration is:
[330]	training's auc: 0.790406	training's binary_logloss: 0.203196	valid_1's auc: 0

In [92]:
predictions_test_lgb

array([0.08618244, 0.04999798, 0.05012544, ..., 0.06365534, 0.05095845,
       0.05267544])

In [93]:
predictions_test_lgb.shape

(261477,)

In [94]:
X_test.shape

(261477, 127)

In [95]:
predictions_train_lgb

array([0.13912319, 0.14868946, 0.1376964 , ..., 0.04106913, 0.04732146,
       0.06919253])

In [96]:
predictions_train_lgb.shape

(260864,)

In [97]:
train_data.shape

(260864, 129)

In [98]:
 all_data_test[all_data_test['label'].isna()].sample()

,user_id,merchant_id,label,prob,age_range,gender,item_path,cat_path,seller_path,brand_path,...,tfidf_90,tfidf_91,tfidf_92,tfidf_93,tfidf_94,tfidf_95,tfidf_96,tfidf_97,tfidf_98,tfidf_99
491220,345380,3941,NaN,NaN,7.0,0.0,529808 931868 931868 238104 310345 131990 3844...,1198 1118 1118 142 766 1118 1173 938 938 1141 ...,2816 2743 2743 2484 4507 2647 211 3941 3941 39...,3186.0 4475.0 4475.0 3929.0 5159.0 2737.0 3851...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [99]:
df_out = pd.DataFrame()
df_out['user_id'] = all_data_test[all_data_test['label'].isna()]['user_id']
df_out['merchant_id'] = all_data_test[all_data_test['label'].isna()]['merchant_id']
df_out['predict_prob'] = predictions_test_lgb


In [100]:
df_out.head()

,user_id,merchant_id,predict_prob
260864,163968,4605,0.086182
260865,360576,1581,0.049998
260866,98688,1964,0.050125
260867,98688,3645,0.050675
260868,295296,3361,0.058642


In [101]:
df_out[df_out['predict_prob']>0.5]

,user_id,merchant_id,predict_prob


In [102]:
df_out['predict_prob'].max()

0.4873157843385426